# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [5]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 101831229,
 'calculationPrice': 'close',
 'change': 1.5,
 'changePercent': 0.00886,
 'close': 166.98,
 'closeSource': 'oiflfaci',
 'closeTime': 1716493715476,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 172.26,
 'delayedPriceTime': 1686060824152,
 'extendedChange': -0.39,
 'extendedChangePercent': -0.0023,
 'extendedPrice': 172,
 'extendedPriceTime': 1678108507492,
 'high': 171.52,
 'highSource': 'uypet en dme5rcild a1ie',
 'highTime': 1696164988793,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 171.86,
 'iexCloseTime': 1674762682872,
 'iexLastUpdated': 1665993718177,
 'iexMarketPercent': 0.014782093864601517,
 'iexOpen': 171.038,
 'iexOpenTime': 1669815775872,
 'iexRealtimePrice': 173.23,
 'iexRealtimeSize': 102,
 'iexVolume': 1391322,
 'lastTradeTime': 1724863349473,
 'latestPrice': 167.28,
 'latestSource': 'Close',
 'latestTime': 'March 21, 2022',
 'latestUpdate': 1689180654796,
 'latestVolume': 971502

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [8]:
pe_ratio = data['peRatio']
pe_ratio

28.65

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        except KeyError:
            pass
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,141.73,36.1,N/A
1,AAL,16.15,-5.42,N/A
2,AAP,213.62,22.14,N/A
3,AAPL,166.01,27.69,N/A
4,ABBV,161.51,25.65,N/A
...,...,...,...,...
497,YUM,121.38,23.01,N/A
498,ZBH,124.09,64.07,N/A
499,ZBRA,448.04,27.65,N/A
500,ZION,68.06,6.84,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [14]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=False, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
# final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1 , inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NOW,576.30,512.97,N/A
1,VTR,61.29,474.09,N/A
2,DXCM,474.43,304.57,N/A
3,KSU,298.21,280.7,N/A
4,FTI,7.75,271.12,N/A
5,ALB,205.96,195.78,N/A
6,SBAC,341.00,157.99,N/A
7,CRM,214.50,145.6,N/A
8,FIS,97.39,144.65,N/A
9,EQIX,726.10,136.14,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [15]:
def portfolio_input():
    global portfolio_size
    while True:
        portfolio_size = input('Enter the size of your portfolio: ')
        try:
            portfolio_size = float(portfolio_size)
            break
        except ValueError:
            print('Please enter a number!')

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [16]:
portfolio_input()

Enter the size of your portfolio: 1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [17]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NOW,576.30,512.97,34
1,VTR,61.29,474.09,326
2,DXCM,474.43,304.57,42
3,KSU,298.21,280.7,67
4,FTI,7.75,271.12,2580
5,ALB,205.96,195.78,97
6,SBAC,341.00,157.99,58
7,CRM,214.50,145.6,93
8,FIS,97.39,144.65,205
9,EQIX,726.10,136.14,27


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [20]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
pb_sales = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [29]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            data[symbol]
        except KeyError:
            continue
            
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A', 
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A',
                ], index=rv_columns
            ),ignore_index=True               
        )
    

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [30]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,183.300,N/A,60.08,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,319.100,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,366.440,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.81,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,27.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.960,N/A,14.43,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.060,N/A,33.51,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,38.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,122.200,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2774.840,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [33]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,140.86,N/A,35.94,N/A,8.28,N/A,6.5000,N/A,24.940698,N/A,12.510281,N/A,N/A
1,AAL,16.51,N/A,-5.22,N/A,-1.50,N/A,0.3692,N/A,-11.734126,N/A,1.226982,N/A,N/A
2,AAP,209.41,N/A,21.79,N/A,4.07,N/A,1.1600,N/A,11.852199,N/A,2.599050,N/A,N/A
3,AAPL,173.08,N/A,28.36,N/A,38.50,N/A,7.2600,N/A,21.635754,N/A,17.011281,N/A,N/A
4,ABBV,165.20,N/A,25.55,N/A,18.45,N/A,5.1600,N/A,12.277607,N/A,9.137676,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.42,N/A,23.56,N/A,-4.27,N/A,5.5900,N/A,20.573009,N/A,9.582898,N/A,N/A
498,ZBH,123.92,N/A,66.09,N/A,2.00,N/A,3.2300,N/A,19.398339,N/A,5.887961,N/A,N/A
499,ZBRA,450.13,N/A,27.55,N/A,7.97,N/A,4.2200,N/A,21.090931,N/A,9.399826,N/A,N/A
500,ZION,69.47,N/A,6.70,N/A,1.52,N/A,3.0400,N/A,5.063699,N/A,2.674383,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [32]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [38]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

# for metric in metrics.keys():
#     for row in rv_dataframe.index:
#         if rv_dataframe.loc[row, metric] == None:
#             rv_dataframe.loc[row, metric] = 0.0
    
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,140.86,N/A,35.94,75.3984,8.28,79.6813,6.5000,80.6773,24.940698,81.8725,12.510281,80.2789,N/A
1,AAL,16.51,N/A,-5.22,3.98406,-1.50,5.77689,0.3692,1.99203,-11.734126,4.78088,1.226982,3.58566,N/A
2,AAP,209.41,N/A,21.79,44.4223,4.07,58.1673,1.1600,14.9402,11.852199,38.4462,2.599050,12.3506,N/A
3,AAPL,173.08,N/A,28.36,61.5538,38.50,96.8127,7.2600,83.2669,21.635754,75.8964,17.011281,90.4382,N/A
4,ABBV,165.20,N/A,25.55,54.7809,18.45,92.8287,5.1600,72.7092,12.277607,40.0398,9.137676,65.5378,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.42,N/A,23.56,49.8008,-4.27,5.37849,5.5900,75,20.573009,72.1116,9.582898,68.5259,N/A
498,ZBH,123.92,N/A,66.09,93.2271,2.00,25.8964,3.2300,49.1036,19.398339,68.1275,5.887961,38.4462,N/A
499,ZBRA,450.13,N/A,27.55,59.761,7.97,78.8845,4.2200,60.757,21.090931,73.1076,9.399826,67.1315,N/A
500,ZION,69.47,N/A,6.70,8.96414,1.52,16.3347,3.0400,46.9124,5.063699,9.56175,2.674383,12.749,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [41]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,140.86,N/A,35.94,75.3984,8.28,79.6813,6.5000,80.6773,24.940698,81.8725,12.510281,80.2789,79.5817
1,AAL,16.51,N/A,-5.22,3.98406,-1.50,5.77689,0.3692,1.99203,-11.734126,4.78088,1.226982,3.58566,4.0239
2,AAP,209.41,N/A,21.79,44.4223,4.07,58.1673,1.1600,14.9402,11.852199,38.4462,2.599050,12.3506,33.6653
3,AAPL,173.08,N/A,28.36,61.5538,38.50,96.8127,7.2600,83.2669,21.635754,75.8964,17.011281,90.4382,81.5936
4,ABBV,165.20,N/A,25.55,54.7809,18.45,92.8287,5.1600,72.7092,12.277607,40.0398,9.137676,65.5378,65.1793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.42,N/A,23.56,49.8008,-4.27,5.37849,5.5900,75,20.573009,72.1116,9.582898,68.5259,54.1633
498,ZBH,123.92,N/A,66.09,93.2271,2.00,25.8964,3.2300,49.1036,19.398339,68.1275,5.887961,38.4462,54.9602
499,ZBRA,450.13,N/A,27.55,59.761,7.97,78.8845,4.2200,60.757,21.090931,73.1076,9.399826,67.1315,67.9283
500,ZION,69.47,N/A,6.70,8.96414,1.52,16.3347,3.0400,46.9124,5.063699,9.56175,2.674383,12.749,18.9044


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [43]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.51,N/A,-5.22,3.98406,-1.5000,5.77689,0.3692,1.99203,-11.734126,4.78088,1.226982,3.58566,4.0239
1,UNM,30.91,N/A,7.58,10.757,0.5421,5.9761,0.5156,3.98406,3.707277,6.57371,0.499847,0.199203,5.49801
2,PRU,119.97,N/A,6.21,7.17131,0.7164,6.97211,0.6164,5.9761,3.919812,7.37052,0.615970,0.796813,5.65737
3,AIG,63.96,N/A,5.82,6.77291,0.7830,7.17131,1.0101,12.1514,3.801634,6.97211,0.997870,2.39044,7.09163
4,GM,44.14,N/A,6.70,8.96414,1.1100,9.56175,0.5280,4.38247,2.307941,5.77689,1.981364,8.96414,7.52988
5,LNC,66.21,N/A,9.12,12.6494,0.5869,6.1753,0.6006,5.37849,6.495940,15.3386,0.587014,0.59761,8.02789
6,MET,71.13,N/A,9.51,13.745,0.8371,7.37052,0.8169,8.36653,5.303393,10.5578,0.779400,1.39442,8.28685
7,F,17.14,N/A,3.80,5.37849,1.4100,14.9402,0.4914,3.38645,4.359727,8.36653,2.314756,10.9562,8.60558
8,C,58.56,N/A,5.74,6.57371,0.6404,6.3745,1.4700,20.7171,3.581057,6.3745,1.324399,4.78088,8.96414
9,SYF,37.71,N/A,5.16,6.1753,1.5300,16.7331,1.1700,15.1394,3.249420,6.1753,1.170753,2.98805,9.44223


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [44]:
portfolio_input()

Enter the size of your portfolio: 2500000


In [46]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.51,3028,-5.22,3.98406,-1.5000,5.77689,0.3692,1.99203,-11.734126,4.78088,1.226982,3.58566,4.0239
1,UNM,30.91,1617,7.58,10.757,0.5421,5.9761,0.5156,3.98406,3.707277,6.57371,0.499847,0.199203,5.49801
2,PRU,119.97,416,6.21,7.17131,0.7164,6.97211,0.6164,5.9761,3.919812,7.37052,0.615970,0.796813,5.65737
3,AIG,63.96,781,5.82,6.77291,0.7830,7.17131,1.0101,12.1514,3.801634,6.97211,0.997870,2.39044,7.09163
4,GM,44.14,1132,6.70,8.96414,1.1100,9.56175,0.5280,4.38247,2.307941,5.77689,1.981364,8.96414,7.52988
5,LNC,66.21,755,9.12,12.6494,0.5869,6.1753,0.6006,5.37849,6.495940,15.3386,0.587014,0.59761,8.02789
6,MET,71.13,702,9.51,13.745,0.8371,7.37052,0.8169,8.36653,5.303393,10.5578,0.779400,1.39442,8.28685
7,F,17.14,2917,3.80,5.37849,1.4100,14.9402,0.4914,3.38645,4.359727,8.36653,2.314756,10.9562,8.60558
8,C,58.56,853,5.74,6.57371,0.6404,6.3745,1.4700,20.7171,3.581057,6.3745,1.324399,4.78088,8.96414
9,SYF,37.71,1325,5.16,6.1753,1.5300,16.7331,1.1700,15.1394,3.249420,6.1753,1.170753,2.98805,9.44223


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [56]:
writer = pd.ExcelWriter('value_strat.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [57]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format': '0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [58]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    # set column formats
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    # reset column title
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [59]:
writer.save()